In [1]:
import pandas as pd
from geopy.distance import geodesic

# Reload the dataset
file_path = (r"C:\Users\riyas\OneDrive - University of Birmingham\Documents\MSc Data Science\2. group project data science\merged_file.csv")
df = pd.read_csv(file_path)
df

,WT_YEAR,WT_LATITUDE,WT_LONGITUDE,WT_COUNTRY,CC_COUNT_POST_WT,CC_LATITUDE,CC_LONGITUDE,CC_OBSERVATION_DATE,CC_DURATION_MINUTES,Distance WT to CC (km),CC_COUNT_PRE_WT
0,2023,28.3553,33.0622,Egypt,20.0,26.845628,33.998315,2023-10-22,90.0,1078.067489,NaN
1,2023,28.3553,33.0622,Egypt,1.0,28.010269,34.408268,2023-12-23,116.0,1047.972563,NaN
2,2023,28.3553,33.0622,Egypt,2.0,27.099550,33.837988,2023-04-01,90.0,140.474153,NaN
3,2023,28.3553,33.0622,Egypt,400.0,27.393946,33.681893,2023-03-14,90.0,949.922860,NaN
4,2023,28.3553,33.0622,Egypt,1.0,27.940700,34.301800,2023-12-24,281.0,721.183116,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1048570,2018,15.2955,76.3177,India,100.0,23.137913,71.768918,2018-01-06,126.0,990.927164,5.0
1048571,2018,15.2955,76.3177,India,100.0,23.207587,71.739006,2018-11-25,180.0,999.148949,5.0
1048572,2018,15.2955,76.3177,India,9.0,23.578610,69.318500,2018-12-09,120.0,1174.582263,5.0
1048573,2018,15.2955,76.3177,India,2.0,28.649528,76.608570,2018-11-19,212.0,1479.041902,5.0


In [5]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree
from geopy.distance import great_circle

In [6]:
# Ensure column names are stripped of leading/trailing spaces
df.columns = df.columns.str.strip()



In [7]:
# Extract wind turbine and common crane coordinates
wind_turbine_coords = np.radians(df[['WT_LATITUDE', 'WT_LONGITUDE']].dropna().values)
crane_coords = np.radians(df[['CC_LATITUDE', 'CC_LONGITUDE']].dropna().values)

In [8]:
# Build KDTree for fast nearest neighbor search
tree = KDTree(wind_turbine_coords)

In [9]:
# Define search radius in radians (10 km)
radius_km = 10
radius_rad = radius_km / 6371.0  # Earth’s radius in km

In [10]:
# Query the KDTree for wind turbines within 10 km of each common crane observation
counts = []
for coord in crane_coords:
    indices = tree.query_ball_point(coord, r=radius_rad)
    counts.append(len(indices))

In [11]:
# Add results to dataframe
df.loc[df[['CC_LATITUDE', 'CC_LONGITUDE']].notna().all(axis=1), 'WT_COUNT_10KM_RADIUS'] = counts

In [12]:
df

,WT_YEAR,WT_LATITUDE,WT_LONGITUDE,WT_COUNTRY,CC_COUNT_POST_WT,CC_LATITUDE,CC_LONGITUDE,CC_OBSERVATION_DATE,CC_DURATION_MINUTES,Distance WT to CC (km),CC_COUNT_PRE_WT,WT_COUNT_10KM_RADIUS
0,2023,28.3553,33.0622,Egypt,20.0,26.845628,33.998315,2023-10-22,90.0,1078.067489,NaN,0.0
1,2023,28.3553,33.0622,Egypt,1.0,28.010269,34.408268,2023-12-23,116.0,1047.972563,NaN,0.0
2,2023,28.3553,33.0622,Egypt,2.0,27.099550,33.837988,2023-04-01,90.0,140.474153,NaN,0.0
3,2023,28.3553,33.0622,Egypt,400.0,27.393946,33.681893,2023-03-14,90.0,949.922860,NaN,0.0
4,2023,28.3553,33.0622,Egypt,1.0,27.940700,34.301800,2023-12-24,281.0,721.183116,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,2018,15.2955,76.3177,India,100.0,23.137913,71.768918,2018-01-06,126.0,990.927164,5.0,0.0
1048571,2018,15.2955,76.3177,India,100.0,23.207587,71.739006,2018-11-25,180.0,999.148949,5.0,0.0
1048572,2018,15.2955,76.3177,India,9.0,23.578610,69.318500,2018-12-09,120.0,1174.582263,5.0,0.0
1048573,2018,15.2955,76.3177,India,2.0,28.649528,76.608570,2018-11-19,212.0,1479.041902,5.0,0.0


In [13]:
# Save updated file
df.to_csv("craneturbine_merged.csv", index=False)